In [37]:
from pathlib import Path
from bs4 import BeautifulSoup
import requests
import lxml
import json
import sqlite3
import pandas as pd
import time
#import asyncio

data_path = Path(Path.cwd(), 'data', 'hw1.db')
url_search = 'https://hh.ru/search/vacancy'
user_agent = {'User-agent': 'Mozilla/5.0'}
url_api = 'https://api.hh.ru/vacancies'
par = {'area': '113'
       ,'text': 'middle python developer'
       ,'search_field': 'description'
       ,'professional_role': '96'
       ,'items_on_page': '20'
       ,'page': '0'
      }
table_name1, table_name2 = 'vacancies1', 'vacancies2'
#Реализация заполнения двух таблиц vacancies1, vacancies2 одинаковой структуры
#вакансиями с hh.ru
#Таблицы должны быть созданы заранее в базе данных SQLite: hw1

In [38]:
#Данные из html-странички, запись в таблицу vacancies1
def first_method():
    links = []
    page = 0
    data_list = []
    #Составляем список 100 ссылок на вакансии
    while (len(data_list) < 100) and (page < 10):
        par['page'] = str(page)
        result = requests.get(url_search, headers = user_agent, params = par)
        if result.status_code != 200:
            page += 1
            continue
        soup = BeautifulSoup(result.content, 'lxml')
    
        links = [l['href'].partition('?')[0] for l in soup.find_all('a', attrs={'data-qa':"serp-item__title"})]
        page += 1
    
        for url_vacancy in links:
            result = requests.get(url_vacancy, headers=user_agent)
            if result.status_code != 200:
                continue
            soup = BeautifulSoup(result.content, 'lxml')
            dic_str = {}
            #Название компании       
            dic_str['company_name'] = soup.find('span', attrs={'data-qa':"bloko-header-2"}).get_text()
            #Название позиции
            dic_str['position'] = soup.find('h1', attrs={'data-qa':"vacancy-title"}).get_text()
            #Описание вакансии
            dic_str['job_description'] = soup.find('div', attrs={'data-qa':"vacancy-description"}).get_text()
            #Ключевые навыки
            dic_str['key_skills'] = '; '.join([s.get_text() for s 
                                               in soup.find_all('span', attrs={'data-qa':"bloko-tag__text"})
                                              ])
            if 'Python' not in dic_str['key_skills']: #Не берём, если отсутствует требование по Python
                continue
            data_list.append(dic_str)
            time.sleep(0.5) #Если делать без паузы, то через много страниц может выдать ошибку страницы
    
    df = pd.DataFrame(data_list)
    try:
        connection = sqlite3.connect(data_path)
    except:
        print(f"Error: Проблема с файлом '{data_path}'")
    try:
        df.to_sql(table_name1, connection, if_exists='replace', index=False)
        print('1st method: ok')
    except:
        print(f"Error: Проблема с записью таблицы {table_name1}")
    connection.close()

In [39]:
#Данные по api
def second_method():
    links = []
    page = 0
    data_list = []
    #Составляем список 100 ссылок на вакансии
    while (len(data_list) < 100) and (page < 10):
        par['page'] = str(page)
        result = requests.get(url_api, params = par, headers = user_agent)
        if result.status_code != 200:
            page += 1
            continue
        result_json = result.json()
        links = [l['url'].split('?')[0] for l in result_json['items']]
        page += 1
    
        for url_vacancy in links:
            result2 = requests.get(url_vacancy, headers = user_agent)
            if result.status_code != 200:
                continue
    
            result2_json = result2.json()
            dic_str = {}
            #Название компании       
            dic_str['company_name'] = result2_json['employer']['name']
            #Название позиции
            dic_str['position'] = result2_json['name']
            #Описание вакансии
            dic_str['job_description'] = BeautifulSoup(result2_json['description'], 'lxml').get_text()
            #Ключевые навыки
            dic_str['key_skills'] = '; '.join([s['name'] for s in result2_json["key_skills"]])
            if 'Python' not in dic_str['key_skills']:  #Не берём, если отсутствует требование по Python             
                continue
            data_list.append(dic_str)
            time.sleep(0.5) #Если делать без паузы, то через много страниц может выдать ошибку страницы
    
    df = pd.DataFrame(data_list)
    try:
        connection = sqlite3.connect(data_path)
    except:
        print(f"Error: Проблема с файлом '{data_path}'")
    try:
        df.to_sql(table_name2, connection, if_exists='replace', index=False)
        print('2nd method: ok')
    except:
        print(f"Error: Проблема с записью таблицы {table_name2}")
    connection.close()

2nd method: ok


In [36]:
first_method()
second_method()